#### Import Python and SQL Alchemy Libraries

In [ ]:
#Python Libraries
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime 
import time

In [ ]:
# SQL Alchemy Libraries
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

#### Create the engine from the database path 

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

#### Reflect an existing database into a new model

In [ ]:
Base = automap_base()

#### Reflect the table and create a session link

In [ ]:
Base.prepare(engine, reflect=True)
session = Session(engine)

#### Viewing all of the classes that automap found

In [ ]:
Base.classes.keys()

#### Save references to each table

In [ ]:
Station = Base.classes.station
Measurement = Base.classes.measurement

## Exploratory Climate Analysis

### Design a query to retrieve the last 12 months of precipitation data and plot the results

#### Order the query by descending order to find the most recent date

In [ ]:
Recent_Date = session.query(Measurement.date).order_by(Measurement.date.desc()).limit(10).all()
Last_Date = Recent_Date[0][0]

#### Find the value 12 months before the most recent date to get the range of interest

In [ ]:
Last_Unix = time.mktime(datetime.datetime.strptime(Last_Date,"%Y-%m-%d").timetuple())
First_Unix = round(Last_Unix - 31500000)
First_Date = datetime.datetime.fromtimestamp(First_Unix).strftime('%Y-%m-%d')

#### Filter the data to show only dates after 08-23-2016 and convert to a pandas dataframe

In [ ]:
Twelve_Month_Query = pd.DataFrame(session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= First_Date).all(),
                                 columns = ['Date','Precipitation'])

#### Set the index to the date column and sort the dataframe by date

In [ ]:
Twelve_Month_Query.set_index('Date', inplace=True)
Twelve_Month_Query_df = Twelve_Month_Query.sort_values(by='Date',ascending=True)

#### Use Pandas Plotting with Matplotlib to plot the data and print a summary of the dataframe

In [ ]:
Twelve_Month_Query_df.plot()
plt.title('Precipitation Over a Period of 12 Months')
plt.xlabel('Date')
plt.ylabel('Inches')

In [ ]:
Twelve_Month_Query_df.describe()

### Design a query to show how many stations are available in this dataset?

In [174]:
Station_Count = session.query(Measurement.station).group_by(Measurement.station).count()
print(f"The number of stations are {Station_Count}")

The number of stations are 9


### What are the most active stations?

In [172]:
Station_list = session.query(Measurement.station).group_by(Measurement.station).all()
stations = [] 
station_count = []

for i in Station_list:
    i_dict = i._asdict()  # sqlalchemy.util._collections.result , has a method called _asdict()
    stations.append(i_dict)
    
for index in range(len(stations)): 
    station_count.append(session.query(Measurement).filter(Measurement.station == stations[index]['station']).count())